# IASA - Champion - Final


# Інсталювання необхідних бібліотек

In [26]:
!pip install inference-sdk
!pip install easyocr
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


завантажуємо зіп файл для розпакування

In [3]:
import zipfile
import os

# Specify the path to the zip archive
zip_file_path = 'IASA_Champ_Final.zip'

# Specify the directory where you want to extract the files
extract_dir = 'IASA_Champ_Final'

# Create the extract directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Open the zip archive
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extract_dir)

print("Zip archive extracted to", extract_dir)

Zip archive extracted to IASA_Champ_Final


This Python script is designed to navigate and process a specific dataset structure, particularly one that organizes application data with screenshots and, optionally, their corresponding accessibility tree JSON files. Here's a step-by-step breakdown of what the code does:

1. **Configuration for OCR**: It sets up the path to the Tesseract-OCR executable, which is necessary for the Optical Character Recognition (OCR) part of the script. OCR is used to extract text from images.

2. **Accessibility Tree Parsing**: It defines a function, `parse_accessibility_tree`, which is intended to read and process the accessibility tree JSON files. These files typically contain structured data about the UI elements on the screen, such as buttons, labels, and other interactive elements. The actual parsing logic inside this function would need to be implemented based on specific requirements, as it's currently a placeholder to demonstrate where and how you'd interact with these files.

3. **Screenshot Processing with OCR**: The `process_screenshot_with_ocr` function is where the script processes images. It reads screenshots, converts them to grayscale, applies a threshold to highlight text and UI elements, and then uses OCR to extract any text present in the image. It also identifies contours (which could correspond to UI elements) and outlines them, essentially attempting to detect and visualize elements on the screen.

4. **Dataset Processing**: The `process_dataset` function is the core of the script, designed to navigate through the dataset's directory structure. It:
   - Iterates through each application's directory within the dataset.
   - For each application, it further iterates through directories that represent different states or screens of the application.
   - Within each state's directory, it looks for both screenshot images (`.png` files) and accessibility tree JSON files.
   - If an accessibility tree JSON file is found, it calls `parse_accessibility_tree` to handle it. This part is crucial for extracting structured data about the UI elements present in the screenshot.
   - Regardless of whether an accessibility tree is available, it processes every screenshot found using the `process_screenshot_with_ocr` function to apply OCR and basic element detection.

5. **Example Usage**: Finally, the script concludes with an example call to `process_dataset`, specifying the path to the dataset. This part of the code needs to be adjusted to reflect the actual path where the `IASA_Champ_Final` dataset is located on your system.



Імпортуємо необхідні бібліотеки

In [33]:
import os
import json
import numpy as np
from inference_sdk import InferenceHTTPClient
import easyocr
from PIL import Image, ImageDraw
import csv
from openai import OpenAI
import openai
import pandas as pd

Початкові налаштування Оптичного Читача

In [13]:
# Initialize the inference client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="Your API key"
)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

root_dir = 'IASA_Champ_Final/IASA_Champ_Final/app_data/'

# Define the directory for saving JSON files
json_save_dir = 'saved_json_files'

Обробка зображень та збереження результатів OCR

In [14]:
# Function to process images and save OCR results in a JSON file
def process_images_with_model(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.png'):
                image_path = os.path.join(subdir, file)
                print(f"Processing {image_path}")

                # Send the image to the inference API
                result = CLIENT.infer(image_path, model_id="ui-elemebts/1")

                # Open the image
                image = Image.open(image_path)

                # Initialize a list to hold OCR results for the current image
                ocr_results = []

                # Iterate over the predictions and apply OCR to each detected UI element
                for element in result['predictions']:
                    if 'x' in element and 'y' in element:
                        x = element['x']
                        y = element['y']
                        width = element['width']
                        height = element['height']

                        # Calculate bounding box for the element
                        bbox = (x - width / 2, y - height / 2, x + width / 2, y + height / 2)
                        element_image = image.crop(bbox)
                        np_array = np.array(element_image)
                        ocr_result = reader.readtext(np_array)

                        # Add OCR result for this element to the list
                        element_ocr = {"x": x, "y": y, "class": element['class'], "text": [text for _, text, _ in ocr_result]}
                        ocr_results.append(element_ocr)

                # Construct the JSON filename and save path
                relative_path = os.path.relpath(image_path, directory)
                json_filename = os.path.splitext(relative_path)[0] + "_ocr_results.json"
                json_save_path = os.path.join(json_save_dir, json_filename)
                os.makedirs(os.path.dirname(json_save_path), exist_ok=True)

                # Save OCR results to a JSON file
                with open(json_save_path, 'w') as json_file:
                    json.dump(ocr_results, json_file, indent=4)

                print(f"OCR results saved to {json_save_path}")

In [15]:
# Start processing
process_images_with_model(root_dir)

Processing IASA_Champ_Final/IASA_Champ_Final/app_data/Receipts/1707209585/Receipts-1707209586.59.png
OCR results saved to saved_json_files/Receipts/1707209585/Receipts-1707209586.59_ocr_results.json
Processing IASA_Champ_Final/IASA_Champ_Final/app_data/Receipts/1707209474/Receipts-1707209475.45.png
OCR results saved to saved_json_files/Receipts/1707209474/Receipts-1707209475.45_ocr_results.json
Processing IASA_Champ_Final/IASA_Champ_Final/app_data/Receipts/1707209532/Receipts-1707209533.36.png
OCR results saved to saved_json_files/Receipts/1707209532/Receipts-1707209533.36_ocr_results.json
Processing IASA_Champ_Final/IASA_Champ_Final/app_data/Receipts/1707209308/Receipts-1707209309.61.png
OCR results saved to saved_json_files/Receipts/1707209308/Receipts-1707209309.61_ocr_results.json
Processing IASA_Champ_Final/IASA_Champ_Final/app_data/Elephas/1707210124/Elephas-1707210125.42.png
OCR results saved to saved_json_files/Elephas/1707210124/Elephas-1707210125.42_ocr_results.json
Processin

Збереження JSON-файлів

In [19]:
# !zip -r /content/saved_json_files.zip /content/saved_json_files

In [20]:
# from google.colab import files
# files.download("/content/saved_json_files.zip")

# Json Formatting

In [ ]:
#json
import json

def format_element(element, depth=0):
    """Recursively formats each element into a simplified tree-like structure, using role_description if value is 'None'."""
    # Determine the type and name of the element
    element_type = element.get('role', 'Unknown').replace('AX', '')
    name = element.get('name', '')
    display_name = f"{name} " if name else ""

    # Prepare the bounding box
    bbox = element.get('bbox', [])
    bbox_str = f"[{', '.join(map(str, bbox))}]" if bbox else ""

    # Decide between value and role_description
    value = element.get('value')
    role_description = element.get('role_description', '')
    value_or_role_desc = role_description if value == "None" else value
    value_str = f": \"{value_or_role_desc}\"" if value_or_role_desc else ""

    # Prepare the indentation
    indent = "    " * depth

    # Combine all parts
    formatted_line = f"{indent}{display_name}{element_type} {bbox_str}{value_str}\n"

    # Process children if any
    children = element.get('children', [])
    for child in children:
        formatted_line += format_element(child, depth + 1)

    return formatted_line

def beautify_json_structure(input_json, output_file):
    """Reads a JSON structure and writes its simplified tree structure to a text file, using role_description if value is 'None'."""
    with open(input_json, 'r') as file:
        data = json.load(file)

    formatted_data = format_element(data)

    with open(output_file, 'w') as file:
        file.write(formatted_data)


In [ ]:
# Example usage
input_json_filename = 'com.hegenberg.bettertouchtool-setapp (1).json'  # Update this to your JSON file path
output_txt_filename = 'outputbetterrouchtool.txt'  # The output file where you want to save the tree structure
beautify_json_structure(input_json_filename, output_txt_filename)
print(open(output_txt_filename).read())


BetterTouchTool Configuration Window [0, 0, 1770, 1000]: "standard window"
    <<< Button [150, 50, 70, 34]: "button"
    Button [2, 926, 76, 74]: "button"
    Button [64, 926, 76, 74]: "button"
    None ScrollArea [0, 94, 370, 836]: "scroll area"
        None Table [0, 94, 370, 836]: "table"
            None Row [0, 94, 370, 52]: "table row"
                None Cell [2, 96, 360, 48]: "cell"
                    None Image [16, 100, 38, 38]: "image"
                    None StaticText [62, 106, 296, 28]: "All Apps"
            None Row [0, 146, 370, 52]: "table row"
                None Cell [2, 148, 360, 48]: "cell"
                    None Image [16, 152, 38, 38]: "image"
                    None StaticText [62, 158, 296, 28]: "Finder"
            None Column [0, 94, 366, 836]: "column"
    None Group [370, 0, 1400, 94]: "group"
        Button [372, 20, 80, 64]: "button"
        None PopUpButton [438, 28, 476, 50]: " • Touch Bar"
        Button [1644, -6, 92, 102]: "button"
        P

In [ ]:
input_json_filename = 'com.apple.clock.json'  # Update this to your JSON file path
output_txt_filename = 'outputclock.txt'  # The output file where you want to save the tree structure
beautify_json_structure(input_json_filename, output_txt_filename)
print(open(output_txt_filename).read())

Clock Window [0, 0, 2048, 1536]: "standard window"
    None Group [0, 0, 2048, 1536]: "group"
        None Group [0, 0, 2048, 1536]: "group"
            None Group [0, 0, 2048, 1536]: "group"
                None Group [0, 0, 2048, 1536]: "group"
                    None Group [0, 104, 2048, 1432]: "collection"
                        None Button [30, 134, 372, 360]: "button"
                            None StaticText [78, 184, 208, 94]: "text"
                            None StaticText [78, 282, 70, 32]: "text"
                            None StaticText [78, 322, 172, 32]: "text"
                            None StaticText [78, 362, 206, 32]: "text"
                            None CheckBox [74, 404, 84, 50]: "1"
                        None Button [434, 134, 372, 360]: "button"
                            None StaticText [482, 184, 180, 94]: "text"
                            None StaticText [482, 282, 70, 32]: "text"
                            None StaticText [482, 322, 292, 32]

In [ ]:
import re

# Example usage

# Ваша строка
input_string = ""

# Регулярное выражение для поиска первых слов до "Window"
regex_pattern = re.compile(r'^([^W]+Window)')

# Ищем соответствие в строке
match = regex_pattern.search(input_string)

# Если найдено, берем первую группу
if match:
    first_words = match.group(1)
else:
    first_words = ""

# Заменяем начало файла
output_string = f'"Name of the app: {first_words}"\n{input_string}'

# Выводим результат
print(output_string)

# Використовуємо OpenAI API

In [37]:

# Assuming PROMPTS is defined somewhere
PROMPTS = json.load(open("prompts.json"))

# Load train_test_split.csv
train_images = []
with open('IASA_Champ_Final/IASA_Champ_Final/train_test_split.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['split'] == 'train':
            train_images.append(row['screen_path'].strip('/'))

# Load ui_questions_train.tsv
questions = []
with open('IASA_Champ_Final/IASA_Champ_Final/ui_questions_train.tsv', newline='') as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    for row in reader:
        if row['Screen id'] in train_images:
            questions.append(row)

# Function to find JSON file for a given screen id
def find_json_for_screen(screen_id):
    for subdir, dirs, files in os.walk('saved_json_files'):
        for file in files:
            if file.startswith(screen_id) and file.endswith('_ocr_results.json'):
                return os.path.join(subdir, file)
    return None


# Configure your OpenAI API key here
openai_client = OpenAI(api_key="Your API key here")

def get_model_response(client, question_type, question, screen_representation):
    prompt = PROMPTS[question_type]
    formatted_prompt = prompt\
        .replace("<screen_representation>", screen_representation)\
        .replace("<question>", question)

    model_response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        temperature=0,
        messages=[{
            "role": "user",
            "content": formatted_prompt
        }],
    )

    answer = json.loads(model_response.choices[0].message.content)["answer"]
    return answer


In [38]:
# Example usage
# Assuming `client` is an instance of `openai.Completion` or similar based on your setup
# This example directly uses `openai` instead of a `client` variable for simplicity

screen_representation = "[{'x': 100, 'y': 200, 'class': 'button', 'text': 'Submit'}]"
question = "How many buttons are on the screen?"
question_type = "number"  # This is just an example; adjust according to your actual data

# Call the function with a dummy client since this example uses `openai` directly
answer = get_model_response(openai, question_type, question, screen_representation)

print(f"Model's response: {answer}")

Model's response: 1


In [39]:

# Process each question for the training images
for question in questions:
    json_file_path = find_json_for_screen(question['Screen id'])
    if json_file_path:
        with open(json_file_path, 'r') as json_file:
            screen_representation = json.load(json_file)
            # Assume screen_representation needs to be formatted as a string
            screen_representation_str = json.dumps(screen_representation)  # Adjust as needed

            answer = get_model_response(openai_client, question['Answer Type'], question['Question'], screen_representation_str)
            print(f"Question: {question['Question']}")
            print(f"GPT's Answer: {answer}")
            print(f"Correct Answer: {question['Answer']}")
            # Add your comparison logic here